In [1]:
## Semaphore

import threading
from  time import sleep
sem = threading.Semaphore()

def fun1():
    print("Fun1 Starting")
    sem.acquire()
    for loop in range(1,5):
        print("Fun1 Working {}".format(loop))
        sleep(1)
    sem.release()
    print("Fun1 Finished")

def fun2():
    print("Fun2 Starting")
    while not sem.acquire(blocking=False):
        print("Fun2 No Semaphore available")
        sleep(1)
    else:
        print("Got Semphore")
        for loop in range(1, 5):
            print("Fun2 Working {}".format(loop))
            sleep(1)
    sem.release()

t1 = threading.Thread(target = fun1)
t2 = threading.Thread(target = fun2)
t1.start()
t2.start()
t1.join()
t2.join()

print("All Threads done Exiting")

Fun1 Starting
Fun1 Working 1
Fun2 Starting
Fun2 No Semaphore available
Fun1 Working 2Fun2 No Semaphore available

Fun1 Working 3
Fun2 No Semaphore available
Fun1 Working 4
Fun2 No Semaphore available
Fun1 FinishedGot Semphore
Fun2 Working 1

Fun2 Working 2
Fun2 Working 3
Fun2 Working 4
All Threads done Exiting


In [ ]:
import asyncio
import time

import aiohttp

START = time.monotonic()


class RateLimiter:
    """Rate limits an HTTP client that would make get() and post() calls.
    Calls are rate-limited by host.
    https://quentin.pradet.me/blog/how-do-you-rate-limit-calls-with-aiohttp.html
    This class is not thread-safe."""
    RATE = 1  # one request per second
    MAX_TOKENS = 10

    def __init__(self, client):
        self.client = client
        self.tokens = self.MAX_TOKENS
        self.updated_at = time.monotonic()

    async def get(self, *args, **kwargs):
        await self.wait_for_token()
        now = time.monotonic() - START
        print(f'{now:.0f}s: ask {args[0]}')
        return self.client.get(*args, **kwargs)

    async def wait_for_token(self):
        while self.tokens < 1:
            self.add_new_tokens()
            await asyncio.sleep(0.1)
        self.tokens -= 1

    def add_new_tokens(self):
        now = time.monotonic()
        time_since_update = now - self.updated_at
        new_tokens = time_since_update * self.RATE
        if self.tokens + new_tokens >= 1:
            self.tokens = min(self.tokens + new_tokens, self.MAX_TOKENS)
            self.updated_at = now


async def fetch_one(client, i):
        url = f'https://httpbin.org/get?i={i}'
        # Watch out for the extra 'await' here!
        async with await client.get(url) as resp:
            resp = await resp.json()
            now = time.monotonic() - START
            print(f"{now:.0f}s: got {resp['args']}")


async def main():
    async with aiohttp.ClientSession() as client:
        client = RateLimiter(client)
        tasks = [asyncio.ensure_future(fetch_one(client, i)) for i in range(20)]
        await asyncio.gather(*tasks)


if __name__ == '__main__':
    # Requires Python 3.7+
    asyncio.run(main())